In [6]:
# !conda install -c conda-forge python-confluent-kafka --yes

In [7]:
from pyarrow.parquet import ParquetFile
from confluent_kafka import Producer
import pyarrow as pa 
import pandas as pd
import json
import time

In [8]:
# Configuration for the bootstraping services
conf = {'bootstrap.servers':'broker:9092'}
producer = Producer(conf)

In [4]:
topic_name = 'lidarOut' # topic name (has to be the tag for parquet files in ../data/raw)
size = 100 # number of rows to be read from parquet file

# Sample with the first "size" rows of dataframe with tag "topic_name"
pf = ParquetFile('../data/raw/202201W1-' + topic_name + '.parquet') 
nrows = next(pf.iter_batches(batch_size = size)) 
df = pa.Table.from_batches([nrows]).to_pandas() 

In [5]:
average_time = df['timestamp'].diff().mean().total_seconds() # Average time between timestamps for entire dataframe

# Publish data
for i in range(df.shape[0]):
    message = df.iloc[i].to_dict()
    message['timestamp']=str(message['timestamp'])
    m = json.dumps(message)
    
    print("Sending: {}".format(message))
    
    producer.produce(topic=topic_name, value= m.encode('utf-8'))
    
    time.sleep(average_time)

producer.flush()

Sending: {'machineID': 'lidarOut', 'tempS1': 22.0, 'tempS2': 1.2, 'distX': -0.215, 'distY': -0.122, 'distZ': -0.168, 'forceX': 227.0, 'forceY': 173.0, 'forceZ': 71.0, 'timestamp': '2022-01-01 00:00:00'}
Sending: {'machineID': 'lidarOut', 'tempS1': 17.599999999999998, 'tempS2': 13.900000000000002, 'distX': -0.298, 'distY': -0.247, 'distZ': -0.195, 'forceX': 134.0, 'forceY': 44.0, 'forceZ': 281.0, 'timestamp': '2022-01-01 00:00:00.050000'}
Sending: {'machineID': 'lidarOut', 'tempS1': 7.1, 'tempS2': 13.900000000000002, 'distX': -0.161, 'distY': -0.166, 'distZ': -0.098, 'forceX': 151.0, 'forceY': 7.0, 'forceZ': 295.0, 'timestamp': '2022-01-01 00:00:00.100000'}
Sending: {'machineID': 'lidarOut', 'tempS1': 1.2, 'tempS2': 13.700000000000001, 'distX': -0.139, 'distY': -0.11, 'distZ': -0.234, 'forceX': 134.0, 'forceY': 193.0, 'forceZ': 103.0, 'timestamp': '2022-01-01 00:00:00.150000'}
Sending: {'machineID': 'lidarOut', 'tempS1': 26.1, 'tempS2': 16.1, 'distX': -0.186, 'distY': -0.059, 'distZ': -

0